In [19]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr

In [20]:
# association recognition - cued recall
with open("simuS2_data/simuS2_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 300]
# df_test = df_test.loc[df_test.session < 300]

In [21]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,pair_idx,list,session,subject
0,809,1001,LEG,PAGE,16,0,0,0
1,489,1310,DOUGH,SIREN,2,0,0,0
2,1232,731,SANDWICH,ICEBERG,12,0,0,0
3,96,259,BASKETBALL,CATERPILLAR,9,0,0,0
4,1232,731,SANDWICH,ICEBERG,12,0,0,0
...,...,...,...,...,...,...,...,...
134995,1480,1422,TOOL,SWITCH,62,2,299,299
134996,1166,613,RANCH,FUNERAL,70,2,299,299
134997,1480,1422,TOOL,SWITCH,62,2,299,299
134998,1166,613,RANCH,FUNERAL,70,2,299,299


In [38]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,pair_idx,type,test,list,session,subject
0,454,943,DETECTIVE,NEPTUNE,0,-1,extra,1,0,0,0
1,329,-1,CLOVE,None,1,10,Item_Pair,1,0,0,0
2,1084,-1,PLAQUE,None,1,20,Different_Item,1,0,0,0
3,484,671,DONKEY,GULL,0,-1,extra,1,0,0,0
4,1452,-1,TERRITORY,None,0,-1,extra,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
107995,957,818,NOSE,LIEUTENANT,0,-1,extra,2,2,299,299
107996,954,347,NITROGEN,COLOGNE,1,75,Item_Pair,2,2,299,299
107997,427,-1,CURTAIN,None,1,78,Pair_Item,2,2,299,299
107998,820,1567,LIGHTNING,VILLAIN,0,-1,extra,2,2,299,299


### Run CMR-IA

In [23]:
s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [24]:
params = cmr.make_default_params()
params.update(learn_while_retrieving = True)

In [25]:
# params.update(
#     beta_enc = 0.4,
#     beta_rec = 0.1, # beta for retrieved item
#     beta_cue = 0.3,
#     beta_rec_post = 0.9,
#     beta_distract = 0.3,
#     gamma_fc = 0.03,
#     s_fc = 0.3,
#     c_thresh_itm = 0.30,
#     c_thresh_ass = 0.50,
#     learn_while_retrieving = True,
#     d_ass = 0.3,
# )
# params

In [26]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting/pso_duplicate")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='S2')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/S2_230928_500-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params

{'beta_enc': 0.221629,
 'beta_rec': 0.412047,
 'beta_cue': 0.191916,
 'beta_rec_post': 0.965246,
 'beta_distract': 0.038655,
 'phi_s': 2,
 'phi_d': 0.5,
 's_cf': 0,
 's_fc': 0.770167,
 'kappa': 0.5,
 'eta': 0.5,
 'omega': 5,
 'alpha': 1,
 'c_thresh': 0.5,
 'c_thresh_itm': 0.819769,
 'c_thresh_ass': 0.829104,
 'd_ass': 0.613294,
 'lamb': 0.5,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0.011789,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.204247,
 'gamma_cf': 0.5}

In [27]:
# params.update(thresh_sigma = 0)

In [28]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode="Recog-Recog")
df_simu['test'] = df_test['test']
df_simu = df_simu.merge(df_test,on=['session','list','test','test_itemno1','test_itemno2'])
df_simu

CMR2 Time: 341.6345765590668


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject
0,0,0,454,943,0.0,6.828294,0.518954,1,DETECTIVE,NEPTUNE,0,-1,extra,0
1,0,0,329,-1,1.0,842.810120,0.879801,1,CLOVE,None,1,10,Item_Pair,0
2,0,0,1084,-1,1.0,1007.208557,0.870891,1,PLAQUE,None,1,20,Different_Item,0
3,0,0,484,671,0.0,2484.385010,0.813789,1,DONKEY,GULL,0,-1,extra,0
4,0,0,1452,-1,0.0,3.632416,0.487395,1,TERRITORY,None,0,-1,extra,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,299,2,957,818,0.0,117.600372,0.661265,2,NOSE,LIEUTENANT,0,-1,extra,299
107996,299,2,954,347,1.0,787.578369,0.883190,2,NITROGEN,COLOGNE,1,75,Item_Pair,299
107997,299,2,427,-1,1.0,2337.998291,0.828785,2,CURTAIN,None,1,78,Pair_Item,299
107998,299,2,820,1567,0.0,407.198822,0.723365,2,LIGHTNING,VILLAIN,0,-1,extra,299


In [29]:
# get correctness and condition
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
def get_cond(x):
    this_type = x['type']
    target = x['correct_ans']
    if target == 1:
        if this_type == "Different_Item":
            return "Different_Item"
        elif this_type == "Item_Pair":
            return "Item_Pair"
        elif this_type == "Pair_Item":
            return "Pair_Item"
        elif this_type == "Same_Item":
            return "Same_Item"
        elif this_type == "Intact_Pair":
            return "Intact_Pair"
    elif target == 0:
        if this_type == "extra":
            return "NR_Lure"
        elif this_type == "Same_Item" or this_type == "Intact_Pair":
            return "Repeated_Lure"
        else:
            return "Discard"
df_simu['condition'] = df_simu.apply(get_cond,axis=1)
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject,correct,condition
0,0,0,454,943,0.0,6.828294,0.518954,1,DETECTIVE,NEPTUNE,0,-1,extra,0,True,NR_Lure
1,0,0,329,-1,1.0,842.810120,0.879801,1,CLOVE,None,1,10,Item_Pair,0,True,Item_Pair
2,0,0,1084,-1,1.0,1007.208557,0.870891,1,PLAQUE,None,1,20,Different_Item,0,True,Different_Item
3,0,0,484,671,0.0,2484.385010,0.813789,1,DONKEY,GULL,0,-1,extra,0,True,NR_Lure
4,0,0,1452,-1,0.0,3.632416,0.487395,1,TERRITORY,None,0,-1,extra,0,True,NR_Lure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,299,2,957,818,0.0,117.600372,0.661265,2,NOSE,LIEUTENANT,0,-1,extra,299,True,NR_Lure
107996,299,2,954,347,1.0,787.578369,0.883190,2,NITROGEN,COLOGNE,1,75,Item_Pair,299,True,Item_Pair
107997,299,2,427,-1,1.0,2337.998291,0.828785,2,CURTAIN,None,1,78,Pair_Item,299,True,Pair_Item
107998,299,2,820,1567,0.0,407.198822,0.723365,2,LIGHTNING,VILLAIN,0,-1,extra,299,True,NR_Lure


In [30]:
df_simu.query("condition == 'Repeated_Lure' and test == 1")

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,pair_idx,type,subject,correct,condition
137,0,1,148,-1,0.0,947.488464,0.765591,1,BONE,None,0,56,Same_Item,0,True,Repeated_Lure
143,0,1,136,-1,0.0,1565.607422,0.790702,1,BLENDER,None,0,59,Same_Item,0,True,Repeated_Lure
247,0,2,441,1471,0.0,272.463043,0.703275,1,DASHBOARD,TOAST,0,85,Intact_Pair,0,True,Repeated_Lure
388,1,0,1275,-1,0.0,575.647827,0.740675,1,SHARK,None,0,28,Same_Item,1,True,Repeated_Lure
531,1,1,815,-1,0.0,203.752121,0.688745,1,LEVER,None,0,55,Same_Item,1,True,Repeated_Lure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107572,298,2,962,848,0.0,274.046814,0.703565,1,NUN,LUMBER,0,89,Intact_Pair,298,True,Repeated_Lure
107672,299,0,266,977,0.0,2642.742432,0.822659,1,CELERY,ONION,0,25,Intact_Pair,299,True,Repeated_Lure
107690,299,0,271,-1,0.0,71.046181,0.636067,1,CERAMICS,None,0,28,Same_Item,299,True,Repeated_Lure
107774,299,1,1033,-1,0.0,880.413147,0.761920,1,PEAR,None,0,56,Same_Item,299,True,Repeated_Lure


In [31]:
# # get pairs and optimal csim
# df_simu['pair'] = df_simu.apply(lambda x: False if x['test_itemno2'] == -1 else True, axis=1)

# csim_two = df_simu.groupby(["pair","correct_ans"]).csim.mean()
# print(csim_two)

# opt_thresh_item = np.mean(csim_two[False])
# opt_thresh_pair = np.mean(csim_two[True])
# print(opt_thresh_item)
# print(opt_thresh_pair)

# def adjust_resp(x):
#     ispair = x['pair']
#     csim = x['csim']
#     if ispair:
#         if csim > opt_thresh_pair:
#             return 1
#         else:
#             return 0
#     else:
#         if csim > opt_thresh_item:
#             return 1
#         else:
#             return 0
# df_simu['s_resp'] = df_simu.apply(adjust_resp, axis=1)
# df_simu

### Analysis

In [32]:
# anal_perform used by pso
import sys
sys.path.append("../Modeling/CMR_IA/fitting/pso_duplicate")
from object_funcs import anal_perform_S2

In [33]:
# def anal_perform_S2(df_simu):

#     # get target items
#     df_target = df_simu.query("condition != 'Discard'")

#     # get pairs data
#     def get_pair(df_tmp):
#         df_tmp_pair = pd.pivot_table(df_tmp,index=["pair_idx","condition"],columns="test",values="correct")
#         df_tmp_pair.columns = ["test1","test2"]
#         df_tmp_pair.reset_index(inplace=True)
#         return df_tmp_pair

#     df_p = df_target.query("condition != 'NR_Lure'")
#     df_pair = get_pair(df_p).reset_index()

#     # get Q values
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
    
#     qs = []
#     conditions = ['Different_Item', 'Item_Pair', 'Pair_Item', 'Intact_Pair', 'Same_Item', 'Repeated_Lure', 'NR_Lure']
#     for cond in conditions:
#         df_tmp = df_pair.query(f"condition == '{cond}'")
#         test2_rsp = pd.Categorical(df_tmp.test2, categories=[0,1])
#         test1_rsp = pd.Categorical(df_tmp.test1, categories=[0,1])
#         df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)

#         try:
#             q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)
#         except:
#             q = 0 if cond == 'NR_Lure' else np.nan

#         qs.append(q)
    
#     # get hit rates and aggregate
#     df_res = pd.DataFrame({'Condition':conditions, 'Q':qs})
#     df_res.set_index('Condition', inplace=True)
#     df_res['Test1_p'] = df_target.groupby(["test","condition"])["s_resp"].mean()[1]
#     df_res['Test2_p'] = df_target.groupby(["test","condition"])["s_resp"].mean()[2]
#     df_res = df_res[['Test1_p','Test2_p','Q']]
#     stats = df_res.values.tolist()

#     return stats

In [34]:
subjects = np.unique(df_simu.subject)
stats = []
for subj in subjects:
    df_subj = df_simu.query(f"subject=={subj} and list % 3 != 0") # discard first list
    # df_subj = df_simu.query(f"subject=={subj}")
    stats_subj = anal_perform_S2(df_subj)
    stats.append(stats_subj)
stats_mean = np.nanmean(stats, axis=0)
stats_mean.round(3)

array([[0.797, 0.777, 0.232],
       [0.804, 0.912, 0.621],
       [0.914, 0.825, 0.605],
       [0.795, 0.837, 0.842],
       [0.92 , 0.931, 0.791],
       [0.108, 0.141, 0.62 ],
       [0.101, 0.099, 0.   ]])

In [35]:
ground_truth = np.array([[0.82, 0.68, 0.26], 
                         [0.82, 0.85, 0.64],
                         [0.91, 0.85, 0.59],
                         [0.81, 0.82, 0.86],
                         [0.90, 0.92, 0.94],
                         [0.07, 0.15, 0.54],
                         [0.07, 0.06, 0]])
err = np.mean(np.power(stats_mean-ground_truth,2))
err

0.0023884287765547994

In [36]:
import scipy as sp
print(sp.stats.sem(np.array(stats),axis=0, nan_policy='omit').round(3))

[[0.008 0.008 0.027]
 [0.006 0.004 0.02]
 [0.005 0.007 0.019]
 [0.008 0.007 0.01]
 [0.007 0.006 0.009]
 [0.012 0.013 0.018]
 [0.002 0.003 0.0]]
